# 02 - Створіть візуалізації, котрі відповідають на поставлені питання

Ваше завдання створити візуалізації, котрі відповідають на наступні питання:

- Як змінювалась структура генерації електроенергії за роками?

- Як залежить споживання електроенергії від дня року та години доби?

- Як змінюється генерація електроенергії з різних джерел впродовж доби?

- Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

- Як змінюється споживання електроенергії впродовж тижня?

Ви самостійно маєте обрати спосіб візуалізації. Ви також маєте написати короткий супровідний текст до кожної візуалізації, котрий пояснює, чому ви обрали саме цей спосіб презентації даних, які ще альтернативи ви розглядали, та чому зупинились саме на цьому варіанті (які його переваги та недоліки).

Джерело даних: Національна енергетична компанія "Укренерго"

## Підготовка даних

In [ ]:
import pandas as pd
import altair as alt
import numpy as np

In [ ]:
alt.data_transformers.enable('default', max_rows=None)

In [ ]:
df = pd.read_excel("2014-2020.xlsx")
df.head()

In [ ]:
df.rename({'GAES_GEN': 'GAES'}, axis=1, inplace=True)

In [ ]:
df['HOUR'] = df['Час/Дата'].str[:-11].astype(int)
df['HOUR'] = np.where(df['HOUR'] > 9, df['HOUR'], '0' + df['HOUR'].astype(str))
df['HOUR'] = np.where(df['HOUR'].astype(int) > 23, '00', df['HOUR'])

In [ ]:
df['TIME'] = df['Час/Дата'].str[-4:] + '-' + df['Час/Дата'].str[-7:-5] + '-' + df['Час/Дата'].str[-10:-8] + ' ' + df['HOUR'].astype(str)

In [ ]:
df['TIME'] = df['TIME'].astype('datetime64[ns]')

In [ ]:
df = pd.melt(df, id_vars=['TIME', 'CONSUMPTION'], 
             value_vars=['AES', 'TEC', 'VDE', 'TES', 'GES', 'GAES'],
            var_name='STRUCT_NAME', value_name='STRUCT_VAL')

In [ ]:
conditions = [
    (df['TIME'].dt.month == 1) | (df['TIME'].dt.month == 2) | (df['TIME'].dt.month == 12),
    (df['TIME'].dt.month == 3) | (df['TIME'].dt.month == 4) | (df['TIME'].dt.month == 5),
    (df['TIME'].dt.month == 6) | (df['TIME'].dt.month == 7) | (df['TIME'].dt.month == 8),
    (df['TIME'].dt.month == 9) | (df['TIME'].dt.month == 10) | (df['TIME'].dt.month == 11)
]

values = ['Winter', 'Spring', 'Summer', 'Autumn']

df['SEASON'] = np.select(conditions, values)

df.head()

In [ ]:
df.info()

In [ ]:
textgrey = '#333333'
ticksgray = '#999999'
lightgray = '#E7E7E7'

## Як змінювалась структура генерації електроенергії за роками?

Складові генерації: АЕС,ТЕЦ,ВДЕ,ТЕС,ГЕС,ГАЕС.

Для цієї візуалізації я обрала bar chart, бо він може гарно передати відношення кількості згенерованої енергії з одного джерела до всієї кількості згенерованої енергії за рік.


In [ ]:
alt.Chart(df,
         title = 'Зміна структури генерації електроенергії за роками'
         ).mark_bar(size=60).encode(
    x = alt.X('year(TIME):O',
              title = None,
              axis = alt.Axis(labelAngle = 0,
                              labelAlign = 'center',
                              labelPadding = 10.0,
                              domain = False,
                              tickColor = lightgray,            
                             ),
            ),
    y = alt.Y('STRUCT_VAL:Q', 
              aggregate = 'sum',
              title = 'К-сть енергії в МВт',
              axis = alt.Axis(domain = False, 
                              labelPadding = 10.0, 
                              tickColor = lightgray,
                              titleColor = textgrey,
                             gridDash = [2, 2])
             ),
    color = alt.Color('STRUCT_NAME:N', title = 'Складові генерації'),
    tooltip = [
    alt.Tooltip(title = 'Складова', field = 'STRUCT_NAME', type = 'nominal'),
    alt.Tooltip(title = 'К-сть енергії в МВт', field = 'STRUCT_VAL', type = 'quantitative', aggregate = 'sum'),
    ]
).properties(width = 700, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=0, 
                  dy=-20, 
                  color = textgrey, 
                  fontWeight=600, 
                  fontSize=14
)

Однак на такому графіку важко порівняти к-сть енергії з одного і того самого джерела за різні роки, тому я зробила ще одну візуалізаію, на якій якраз можна побачити і порівняти, як змінювалася к-сть енергії одного і того ж джерела протягом різних років.

In [ ]:
alt.Chart(df,
         title = 'Зміна структури генерації електроенергії за роками'
         ).mark_bar().encode(
    x = alt.X('year(TIME):O',
              title = None,
              axis = alt.Axis(labelAngle = 0,
                              labelAlign = 'center',
                              labelPadding = 10.0,
                              domain = False,
                              tickColor = ticksgray
                             ),
             ),
    y = alt.Y('STRUCT_VAL:Q', 
              aggregate = 'sum',
              title = 'К-сть енергії в МВт',
              axis = alt.Axis(domain = False, 
                              labelPadding = 10.0, 
                              tickColor = ticksgray,
                              titleColor = textgrey,
                              gridDash = [2, 2]
                             )
             ),
    color = alt.Color('STRUCT_NAME:N', title = 'Складові генерації'),
    tooltip = [
    alt.Tooltip(title = 'Складова', field = 'STRUCT_NAME', type = 'nominal'),
    alt.Tooltip(title = 'К-сть енергії в МВт', field = 'STRUCT_VAL', type = 'quantitative', aggregate = 'sum'),
    ],
    column = alt.Column('STRUCT_NAME:N', title = None),
).properties(width = 750/len(pd.unique(df.STRUCT_NAME)), height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=340, dy=-20, 
                  color = textgrey, 
                  fontWeight=600, 
                  fontSize=14)

## Як залежить споживання електроенергії від дня року та години доби?

В цій візуалізації потрібно передати три величини, дві з яких час. Для цього може підійти heatmap. На вісях X та Y будуть часові проміжки, а кількість спожитої енерггії буде передаватися кольором.

Оскільки у таблиці присутні дані за декілька років, кожна позначка на графіку буде середнім арефметичним, характерним для певної години певного дня.

Мінус такого представлення в тому, що у нас буде 365 значень на осі Х, тому буде вадко роздивитися кожен окремий елемент, але ми зможемо подивитися на картину в цілому.

In [ ]:
alt.Chart(df,
         title={
                "text": ["Залежність споживання електроенергії від дня року та години доби"], 
                "subtitle": ["(кожна позначка на графіку - середнє арефметичне за 2014-2020рр, характерне для певної години певного дня)"],
                }
         ).mark_rect().encode(
    alt.Y('hoursminutes(TIME):O', title=None,
          axis = alt.Axis(tickColor = lightgray, labelPadding = 5.0,),),
    alt.X('monthdate(TIME):O', title=None,
          axis = alt.Axis(ticks = False, labelPadding = 5.0,),
         ),
    alt.Color('CONSUMPTION:Q', 
              title='Споживання в МВт', 
              aggregate = 'average')
).properties(width = 800, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=0, 
                  dy=-10, 
                  color = textgrey, 
                  fontWeight=600, 
                  fontSize=14)

## Як змінюється генерація електроенергії з різних джерел впродовж доби?

In [ ]:
alt.Chart(df,
         title = 'Зміна генерації електроенергії з різних джерел впродовж доби'
         ).mark_bar().encode(
    x = alt.X('hoursminutes(TIME):O',
              title = None,
              axis = alt.Axis(labelAngle = 0,
                              labelAlign = 'center',
                              labelPadding = 10.0,
                              domain = False,
                              ticks = False
              ),
             ),
    y = alt.Y('STRUCT_VAL:Q', 
              aggregate = 'average',
              title = 'К-сть енергії в МВт',
              axis = alt.Axis(domain = False, 
                              labelPadding = 10.0, 
                              tickColor = lightgray,
                              titleColor = textgrey,
                              gridDash = [2, 2])
             ),
    color = alt.Color('STRUCT_NAME:N', title = 'Складові генерації'),
    tooltip = [
    alt.Tooltip(title = 'Складова', field = 'STRUCT_NAME', type = 'nominal'),
    alt.Tooltip(title = 'К-сть енергії в МВт', field = 'STRUCT_VAL', type = 'quantitative', aggregate = 'average'),
    ],
).properties(width = 750, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=0, 
      dy=-20, 
      color = textgrey, 
      fontWeight=600, 
      fontSize=14,
)

## Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

Я розбила це завдання на два графіки, перший стосується місяців, а другий пір року.

In [ ]:
alt.Chart(df,
          title={
                "text": ["Споживання електроенергії впродовж доби у розрізі місяців року"], 
                "subtitle": ["(кожна позначка на графіку - середнє арефметичне за 2014-2020рр, характерне для певної години певного місяця)"],
                }
         ).mark_rect().encode(
    alt.Y('hoursminutes(TIME):O', title=None,
          axis = alt.Axis(labelPadding = 5.0,),
         ),
    alt.X('month(TIME):O',
          title = None,
          axis = alt.Axis(labelAngle = 0,
                          labelAlign = 'center',
                          labelPadding = 10.0,
                         ),
         ),
    alt.Color('CONSUMPTION:Q', title='consumption', aggregate = 'average')
).properties(width = 800, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=0, 
      dy=-10, 
      color = textgrey, 
      fontWeight=600, 
      fontSize=14)

Наступний рафік більш узагальнена версія попереднього. Показує як змінюється споживання електроенергії залежно від години доби та  пори року.

In [ ]:
alt.Chart(df, 
          title={
                "text": ["Споживання електроенергії впродовж доби у розрізі пір року"], 
                "subtitle": ["(кожна позначка на графіку - середнє арефметичне за 2014-2020рр, характерне для певної години певної пори року)"],
                }
         ).mark_rect().encode(
    alt.Y('hoursminutes(TIME):O', 
          title=None,
          
         ),
    alt.X('SEASON:N', 
          title=None, 
          sort=['Spring', 'Summer', 'Autumn', 'Winter'],
          axis = alt.Axis(labelAngle = 0,
                          labelAlign = 'center',
                          labelPadding = 10.0,
                          labelFontSize = 12.0
                         ),
         ),
    alt.Color('CONSUMPTION:Q', title='Споживання', aggregate = 'average'),
    tooltip = [
    alt.Tooltip(title = 'К-сть енергії в МВт', field = 'CONSUMPTION', type = 'quantitative', aggregate = 'average', format = '.4s'),
    ],
).properties(width = 800, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dx=0, 
                  dy=-10, 
                  color = textgrey, 
                  fontWeight=600, 
                  fontSize=14,
)

## Як змінюється споживання електроенергії впродовж тижня?

Для цієї візуалізації я обрала Bar Chart, висота стовпчиків допоможе порівняти кількість споживання електроенергії протягом тижня.

Також я додала кольори, як допоміжну функцію. Основний спосіб порівняння всетаки по висоті стовпчика. Кольори просто допомагають одразу виділити в який день тижня споживання в середньому найменше(Неділля), а в який - найбільше(Четвер).


In [ ]:
alt.Chart(df, title = 'Споживання електроенергії впродовж тижня').mark_bar(size=60).encode(
    x = alt.X('day(TIME):O',
              title = None,
              axis = alt.Axis(labelAngle = 0,
                              labelAlign = 'center',
                              labelPadding = 10.0,
                              domain = False,
                              ticks = False
              ),
             ),
    y = alt.Y('CONSUMPTION:Q', 
              aggregate = 'average',
              title = 'Споживання енергії в МВт (середнє арифметичне)',
              axis = alt.Axis(domain = False, 
                              labelPadding = 10.0, 
                              tickColor = lightgray,
                              titleColor = textgrey,
                              gridDash = [2, 2])
              ),
    color = alt.Color('CONSUMPTION:Q', aggregate = 'average', legend=None),
    tooltip = [
    alt.Tooltip(title = 'К-сть енергії в МВт', field = 'CONSUMPTION', type = 'quantitative', aggregate = 'average', format = '.4s'),
    ],
).properties(width = 750, height = 500, background = '#F9F9F9', padding = 25
).configure_title(dy=-20, 
      color = textgrey, 
      fontWeight=600, 
      fontSize=14)